<a href="https://colab.research.google.com/github/yorkjong/stock-reports/blob/main/notebooks/ibd_reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stock Screening Using IBD's RS Rating Methodology

This notebook demonstrates stock screening techniques inspired by the methodology used in Investor's Business Daily (IBD). It incorporates the (price) RS Rating, EPS RS Rating, and Revenue RS Rating to evaluate stocks based on price performance, earnings, and revenue growth. The Price RS Rating typically reflects a stock’s performance over the past year, but a 3-month version is also available to assist with short-term trading decisions.

## Install and Setup (this section will be executed automatically)

### Install Required Packages

In [1]:
%pip install "git+https://github.com/yorkjong/vistock.git@feature/ranking_utils"
%pip install requests-cache

!wget -O NotoSansTC-Regular.ttf https://share.cole.tw/d/Tools%20-%20MAC/Fonts/Noto_Sans_TC/static/NotoSansTC-Regular.ttf?sign=bATsZP5QZdI_2EM15sAbcAE48Cacle91CpwUNOCMuM8=:0

  Cloning https://github.com/yorkjong/vistock.git (to revision feature/ranking_utils) to /tmp/pip-req-build-i9bcwtdg
  Running command git clone --filter=blob:none --quiet https://github.com/yorkjong/vistock.git /tmp/pip-req-build-i9bcwtdg
  Running command git checkout -b feature/ranking_utils --track origin/feature/ranking_utils
  Switched to a new branch 'feature/ranking_utils'
  Branch 'feature/ranking_utils' set up to track remote branch 'feature/ranking_utils' from 'origin'.
  Resolved https://github.com/yorkjong/vistock.git to commit 31009409e0963904695b41f7247f6a3421f6f731
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.1 MB/s eta 0:00:00
  Created wheel for vistock: filename=vistock-0.8.0-py3-none-any.whl size=89294 sha256=31bc0202d9d61614b44add7def6fc78d72d32bdacd9bd74de4068bfcf716b80b
  Stored in directory: /tmp/pip-ephem-wheel-cache-y7y3v82y/wheels/37/13/2b/a599ed1c9af840bb5c5d20b1aa8e13e50523bc0d554a13952d
Successfully bu

### Setup and Configuration

In [2]:
# @title Enable Requests Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

In [3]:
# @title Set Chinese Font for matplotlib
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

fm.fontManager.addfont('NotoSansTC-Regular.ttf')
font_name = 'Noto Sans TC'
if font_name not in plt.rcParams['font.sans-serif']:
    plt.rcParams['font.sans-serif'].insert(0, font_name)

In [4]:
# @title ReadOnlyGitHub
import requests
import pandas as pd
from io import StringIO

class ReadOnlyGitHub:
    def __init__(self, repo_owner, repo_name, dir='', branch='main'):
        dir = dir.strip('/')
        base = 'https://raw.githubusercontent.com'
        if dir:
            self.raw_url = f'{base}/{repo_owner}/{repo_name}/{branch}/{dir}'
        else:
            self.raw_url = f'{base}/{repo_owner}/{repo_name}/{branch}'

        base = 'https://api.github.com/repos'
        if dir:
            self.api_url = f'{base}/{repo_owner}/{repo_name}/contents/{dir}'
        else:
            self.api_url = f'{base}/{repo_owner}/{repo_name}/contents'
        self.branch = branch

    def file_exists(self, file_path):
        url = f'{self.raw_url}/{file_path}'
        response = requests.head(url)
        return response.status_code == 200

    def list_filenames(self, dir_path=''):
        if dir_path:
            url = f'{self.api_url}/{dir_path}?ref={self.branch}'
        else:
            url = f'{self.api_url}?ref={self.branch}'
        response = requests.get(url)
        if response.status_code == 200:
            files = response.json()
            return [item['name'] for item in files]
        elif response.status_code == 404:
            print(f"Directory not found: {url}")
            return []
        else:
            print(f"Request failed: {response.status_code} - {response.text}")
            return []

    def download_csv(self, file_path):
        url = f'{self.raw_url}/{file_path}'
        if self.file_exists(file_path):
            return pd.read_csv(url)
        else:
            return pd.DataFrame()

# Create a GitHub instance
github = ReadOnlyGitHub(
    repo_owner='yorkjong',
    repo_name='stock-reports',
    dir='ibd',
    branch='data'
)

In [5]:
# @title Taiwan Stock Name Lookup

class StockNameLookup:
    _df = None  # Class-level variable to hold the DataFrame

    @classmethod
    def _load_data(cls):
        if cls._df is None:  # Check if the DataFrame is already loaded
            gh = ReadOnlyGitHub(
                repo_owner='yorkjong',
                repo_name='stock-reports',
                dir='data/stock_list',
            )
            cls._df = gh.download_csv('taiwan_stock_OpenAPI.csv')

    @classmethod
    def tw_stock_name(cls, ticker):
        cls._load_data()  # Ensure data is loaded before accessing

        code = ticker.split('.')[0]  # Extract the code part

        # Filter the DataFrame to find the stock name for the given code
        stock_name = cls._df.loc[cls._df['Code'] == code, 'Name']

        # Check if the stock_name is empty and return an appropriate message
        if not stock_name.empty:
            return stock_name.values[0]  # Return the first matched stock name
        else:
            return None  # Return None if ticker not found


def tw_stock_name(ticker):
    return StockNameLookup.tw_stock_name(ticker)

In [6]:
# @title Metadata

def parse_metadata(filename):
    components = filename.split('_')
    if len(components) != 5:
        raise ValueError("Filename does not have the expected number of components")

    source, kind, period, rs_win, date = components
    rs_win = rs_win.replace('ibd', '')
    date = date.replace('.csv', '')

    return {
        "Source": source,
        "Type": kind,
        "Period": period,
        "RS window": rs_win,
        "Date": date
    }


def print_metadata(meatadata):
    for key, value in meatadata.items():
        print(f"{key}: {value}")

In [7]:
# @title IBD Financial files
import re
from datetime import datetime


def get_latest_file(file_list, source):
    pattern = rf'^{re.escape(source)}_(\w+)_fin_(\d+)\.csv$'
    matching_files = []

    for file in file_list:
        match = re.match(pattern, file)
        if match:
            date_str = match.group(2)
            date = datetime.strptime(date_str, "%Y%m%d")
            matching_files.append((file, date))

    if not matching_files:
        return None

    return max(matching_files, key=lambda x: x[1])[0]


def fin_download_latest_csv(source):
    gh = ReadOnlyGitHub(
        repo_owner='yorkjong',
        repo_name='stock-reports',
        dir='ibd_fin',
        branch='data'
    )
    file_list = gh.list_filenames()
    fin_filename = get_latest_file(file_list, source)
    df = gh.download_csv(fin_filename)
    return df, fin_filename

In [8]:
# @title DataFrame Utilities

def print_column(df, column):
    if column in df.columns:
        print(', '.join(df[column]))

In [9]:
# @title Source of Tickers

def tickers_from_df(df):
    if 'Name' in df.columns:
        return [name.strip() for names in df['Name']
                for name in names.split(',')]
    elif 'Ticker' in df.columns:
        return [ticker.strip() for tickers in df['Ticker']
                for ticker in tickers.split(',')]
    return []

def major_indices():
    return ['^DJI', '^IXIC', '^NDX', '^RUT', '^SOX',
            '^TWII', '^N225', '^HSI',
            '^STOXX50E', '^FTSE', '^GDAXI', '^FCHI', '^GSPTSE']

def sector_indices():
    return ['SOXX', 'DVY',
            'IWB','IWM', 'IWV',  'IJR',
            'ITB', 'IHI', 'IYC', 'ITA', 'IAK',
            'IYZ', 'IYT', 'IYR', 'IYF', 'IYJ',
            'IYG', 'IYH', 'IYK', 'IDU', 'IYE', 'IHE',
            'IAT', 'IAI', 'IEO', 'IYM', 'IHF']

In [10]:
# @title Checkboxes

import ipywidgets as widgets

def cbs_create(symbols, n_pre_checked=10):
    '''Create a list of checkboxes'''
    return [
        widgets.Checkbox(
            value=(i < n_pre_checked),  # Set first n items as checked
            description=symbol,
            layout=widgets.Layout(width='auto'),
            style={'description_width': 'auto'}
        )
        for i, symbol in enumerate(symbols)
    ]

def cbs_with_grid(checkboxes, n_cols=5):
    '''Create a grid layouting the given checkboxes'''
    return widgets.GridBox(checkboxes, layout=widgets.Layout(
        width='auto',
        grid_template_columns=f'repeat({n_cols}, 1fr)',
        grid_gap='10px'  # Add some space between the checkboxes
    ))

def cbs_get_selected(checkboxes):
    '''Get the selected symbols from the given checkboxes'''
    return [checkbox.description for checkbox in checkboxes if checkbox.value]

def cbs_unselect_all(checkboxes):
    '''Unselect all checkboxes in the given list'''
    for checkbox in checkboxes:
        checkbox.value = False

def cbs_select_top(checkboxes, n=10):
    '''Select the top n checkboxes in the given list'''
    for i, checkbox in enumerate(checkboxes):
        checkbox.value = (i < n)

In [11]:
# @title Dropdown Menus

def create_period_dropdown(value='2y'):
    return widgets.Dropdown(
        options=['1y', '2y', '5y'],
        value=value,
        description='Period:',
    )

def create_interval_dropdown(value='1wk'):
    return widgets.Dropdown(
        options=['1d', '1wk'],
        value=value,
        description='Interval:',
    )

def create_rs_window_dropdown(value='3mo'):
    return widgets.Dropdown(
        options=['3mo', '12mo'],
        value=value,
        description='rs_window:',
    )

def create_style_dropdown(desc=None, value=None):
    return widgets.Dropdown(
        options=['default', 'classic', 'yahoo', 'charles', 'tradingview', 'binance', 'binancedark', 'mike', 'nightclouds', 'checkers', 'ibd', 'sas', 'starsandstripes', 'kenan', 'blueskies', 'brasil'],
        value='yahoo' if value is None else value,
        description='Style:' if desc is None else desc,
        style={'description_width': 'initial'},
    )

def create_template_dropdown(desc=None, value=None):
    return widgets.Dropdown(
        options=['plotly', 'plotly_white', 'plotly_dark', 'ggplot2', 'seaborn', 'simple_white', 'presentation', 'xgridoff', 'ygridoff'],
        value='plotly_dark' if value is None else value,
        description='Template:' if desc is None else desc,
        style={'description_width': 'initial'},
    )

In [12]:
# @title Multiple Searchable Dropdown Menus
import ipywidgets as widgets

def create_search_box():
    '''Create a Text widget for search input'''
    return widgets.Text(
        description='Search:',
        placeholder='Type to search',
        layout=widgets.Layout(width='auto')
    )

def create_dropdown(options, description='Stock:'):
    '''Create a Dropdown widget for displaying filtered options'''
    return widgets.Dropdown(
        description=description,
        options=[None] + options,  # None as the default option
        layout=widgets.Layout(width='auto'),
        value=None  # Set default value to None
    )

def update_dropdown(change, dropdown, options):
    '''Update the options in the dropdown based on search input'''
    search_text = change['new'].lower()
    filtered = [option for option in options if search_text in option.lower()]
    if filtered:
        dropdown.options = [None] + filtered
        dropdown.value = filtered[0]  # Auto-select the first matching option
    else:
        dropdown.options = [None]  # Retain only the None option if no match

def remove_duplicates_preserve_order(lst):
    '''Remove duplicates from a list while preserving order'''
    seen = set()
    result = []
    for item in lst:
        if item and item not in seen:
            seen.add(item)
            result.append(item)
    return result

def get_dropdowns_selected_options(dropdowns):
    '''Get selected options from the dropdowns'''
    selected = [dropdown.value for dropdown in dropdowns if dropdown.value]
    return remove_duplicates_preserve_order(selected)

def create_search_dropdowns(options, max_selections):
    '''Create a layout with search boxes and dropdowns'''
    # Create UI components
    search_boxes = [create_search_box() for _ in range(max_selections)]
    dropdowns = [create_dropdown(options) for _ in range(max_selections)]

    # Bind search box and dropdown menu events
    for search_box, dropdown in zip(search_boxes, dropdowns):
        search_box.observe(lambda change, dropdown=dropdown:
                            update_dropdown(change, dropdown, options),
                            names='value')
    # Create the layout
    controls = [widgets.HBox([search_box, dropdown])
                for search_box, dropdown  in zip(search_boxes, dropdowns)]
    layout = widgets.VBox(controls)
    return dropdowns, layout

In [13]:
# @title Outputs
outputs = widgets.VBox()

In [14]:
# @title enable_plotly_in_cell
# ref. https://stackoverflow.com/questions/76593068/plotly-figure-not-rendering-in-ipywidgets-interact-function-google-colab
def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
  init_notebook_mode(connected=False)

### Glossary of Terms

#### Source (The source of stocks to analyze)
- This could include stocks traded on exchanges or components of a specific index.
- Common abbreviation(s) for the exchange or market sector:
  - For Taiwan Markets, possible values include:
    - **`TWSE`**: Taiwan Stock Exchange (台灣上市股票交易所)
    - **`TPEX`**: Taipei Exchange (上櫃交易所)
    - **`ESB`**: Emerging Stock Board (興櫃交易所)
  - Can also be combined with `+` (e.g., `TWSE+TPEX`, `TWSE+TPEX+ESB`).
  - For America Markets, possible values include:
    - **`SPX`**: S&P 500 (標普五百指數)
    - **`DJIA`**: Dow Jones Industrial Average (道瓊指數)
    - **`NDX`**: NASDAQ-100 (納斯達克一百指數)
    - **`SOX`**: PHLX Semiconductor Index (費半指數)
  - Multiple indices can be combined using `+` (e.g., `SPX+DJIA+NDX+SOX`).

#### Period (Historical Data Time Range)
- The time range for which to fetch historical data.
  - **`2y`**: 2 years
  - **`6mo`**: 6 months

#### RS Window (Window for RS Calculation)
- The time window ('3mo' or '12mo') for Relative Strength:
  - **`3mo`**: 3 months
  - **`12mo`**: 12 months

#### RS (Relative Strength)
- Relative Strength (RS) is a metric used to evaluate the performance of a stock relative to a benchmark index.
  - A higher RS rating indicates that the stock has outperformed the index, while a lower RS rating suggests underperformance.
  - A value of 100 represents the performance of the benchmark index or market.
- The IBD RS calculates the performance over the last year, with the most recent quarter weighted double.
- The IBD 3-month RS calculates the performance over the last quarter.

#### RS Rating
- RS Rating, ranging from 1 (worst) to 99 (best), evaluates the price performance of a stock relative to a benchmark index.

# Execute Actions Step by Step


## Step 1. Select and Preview a File

---



In [15]:
# @title Step 1.1 Select a File

import ipywidgets as widgets

with requests_cache.disabled():
    filenames = github.list_filenames()

# Extract and sort all unique values in reverse order
all_dates = sorted(set(fn.split('_')[4].replace('.csv', '') for fn in filenames), reverse=True)
all_sources = sorted(set(fn.split('_')[0] for fn in filenames))
#all_types = sorted(set(fn.split('_')[1] for fn in filenames), reverse=True)
#all_periods = sorted(set(fn.split('_')[2] for fn in filenames))
all_rs_windows = sorted(set(fn.split('_')[3].replace('ibd', '') for fn in filenames))

# Calculate the maximum length of filenames to set dropdown width
if filenames:
    max_filename_length = max(len(fn) for fn in filenames)
else:
    max_filename_length = 0
dropdown_width = f'{max_filename_length * 10}px'  # 10px width per character

# Create dropdowns with dynamic width
date_dropdown = widgets.Dropdown(
    options=all_dates,
    description='Date:',
    layout=widgets.Layout(width=dropdown_width)
)

source_dropdown = widgets.Dropdown(
    options=all_sources,
    description='Source:',
    layout=widgets.Layout(width=dropdown_width)
)

rs_window_dropdown = widgets.Dropdown(
    options=all_rs_windows,
    description='RS window:',
    layout=widgets.Layout(width=dropdown_width)
)

# Create file selection dropdown
file_dropdown = widgets.Dropdown(
    options=[],
    description='File:',
    layout=widgets.Layout(width=dropdown_width)
)

output = widgets.Output()

def update_dropdowns(*args):
    # Filter files based on selected date
    date_filtered_files = [fn for fn in filenames if date_dropdown.value in fn]

    # Update Source dropdown
    available_sources = sorted(set(fn.split('_')[0] for fn in date_filtered_files))
    source_dropdown.options = available_sources
    if source_dropdown.value not in available_sources:
        source_dropdown.value = available_sources[0] if available_sources else None

    # Update RS window dropdown
    available_rs_windows = sorted(set(fn.split('_')[3].replace('ibd', '') for fn in date_filtered_files))
    rs_window_dropdown.options = available_rs_windows
    if rs_window_dropdown.value not in available_rs_windows:
        rs_window_dropdown.value = available_rs_windows[0] if available_rs_windows else None

    # Update file options
    update_file_options()

def update_file_options(*args):
    # Filter files based on selected date, source, and RS window
    filtered_files = [
        fn for fn in filenames
        if (date_dropdown.value in fn and
            source_dropdown.value == fn.split('_')[0] and
            rs_window_dropdown.value == fn.split('_')[3].replace('ibd', ''))
    ]
    file_dropdown.options = filtered_files
    if filtered_files:
        global selected_file, metadata
        file_dropdown.value = filtered_files[0]  # Set initial value to the first match
        selected_file = file_dropdown.value
        metadata = parse_metadata(selected_file)
        with output:
            output.clear_output()
            print_metadata(metadata)
    else:
        file_dropdown.value = None

# Bind event handlers
date_dropdown.observe(update_dropdowns, 'value')
source_dropdown.observe(update_file_options, 'value')
rs_window_dropdown.observe(update_file_options, 'value')

# Display all dropdowns
display(date_dropdown, source_dropdown, rs_window_dropdown, file_dropdown, output)

# Initialize dropdowns
update_dropdowns()

Dropdown(description='Date:', layout=Layout(width='410px'), options=('20241026', '20241025', '20241024', '2024…

Dropdown(description='Source:', layout=Layout(width='410px'), options=('TWSE+TPEX', 'U.S.Listed'), value='TWSE…

Dropdown(description='RS window:', layout=Layout(width='410px'), options=('12mo', '3mo'), value='12mo')

Dropdown(description='File:', layout=Layout(width='410px'), options=(), value=None)

Output()

In [23]:
# @title Step 1.2 Load and Preview the Files
print(f'{selected_file}:')
with requests_cache.disabled():
    df_rs = github.download_csv(selected_file)
display(df_rs)

with requests_cache.disabled():
    df_fin, fn_fin = fin_download_latest_csv(metadata['Source'])
print(f'\n{fn_fin}:')
display(df_fin)

print(f'\nMerged:')
df_rs2merge = df_rs.drop(['1wk:end max', 'Rating (3M:1M)', 'Rating (6M:3M)',
                          'Rating (9M:6M)'], axis=1)

columns_to_keep = ['Ticker', 'Sector', 'Industry',
                   'EPS RS', 'TTM EPS', 'Rev RS', 'TTM RPS', 'TTM PE']
df_fin2merge = df_fin[columns_to_keep]

df_merge = pd.merge(df_rs2merge, df_fin2merge, on='Ticker', how='left')
display(df_merge)

TWSE+TPEX_stocks_1y_ibd3mo_20241026.csv:


,Ticker,Name,RS,1wk:end max,1mo:1wk max,3mo:1mo max,6mo:3mo max,9mo:6mo max,Rating (RS),Rating (3M:1M),Rating (6M:3M),Rating (9M:6M),Price,52W pos,MA50,MA200,Volume / VMA50
0,6144,得利影,286.30,314.94,334.85,335.79,196.65,112.51,99,99,98,81,85.1,0.66,104.29,47.32,0.25
1,3230,錦明,284.60,288.67,287.13,237.68,138.77,105.71,99,99,93,71,48.9,0.74,48.43,22.35,0.61
2,8937,合騏,236.90,236.90,236.51,205.24,108.28,92.42,99,98,73,12,83.9,1.00,65.57,34.32,0.96
3,4510,高鋒,213.59,223.80,238.85,227.27,108.10,96.92,99,99,73,37,45.5,0.76,45.86,25.66,0.54
4,4583,台灣精銳,211.83,211.83,200.16,166.94,107.82,99.09,99,97,73,48,802.0,0.95,626.31,348.01,1.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019,3228,金麗科,70.04,70.04,72.79,84.79,170.77,240.51,1,5,97,99,223.5,0.24,247.49,361.54,0.81
2020,8088,品安,69.34,69.34,70.00,83.22,95.05,108.72,1,4,42,76,26.7,0.11,28.27,39.42,0.31
2021,8477,創業家,67.95,69.94,77.51,96.93,95.30,113.32,1,48,43,82,12.7,0.13,12.14,17.37,3.05
2022,8085,福華,66.27,66.57,69.04,93.75,136.45,152.94,1,34,92,97,36.8,0.09,42.01,59.64,1.32



TWSE+TPEX_stocks_fin_20241019.csv:


,Ticker,Name,Sector,Industry,Price,EPS QoQ (%),QoQ 2Q Algo (%),QoQ 3Q Algo (%),EPS YoY (%),YoY 2Q Algo (%),EPS RS,TTM EPS,Rev RS,TTM RPS,TTM PE,Rating (EPS RS),Rating (Rev RS)
0,2724,藝舍-KY,Consumer Cyclical,Lodging,36.00,3.71,-1.30,-4.32,31000000.00,-13.00,1.033333e+09,-0.390,15.69,2.719,NaN,99.0,90.0
1,2369,菱生,Technology,Semiconductor Equipment & Materials,18.45,2.08,-0.72,-2.62,27000000.00,0.00,9.000000e+08,-0.560,-27.46,15.287,NaN,99.0,30.0
2,3349,寶德,Technology,Computer Hardware,26.00,4000.68,-0.05,-0.06,4320.65,0.38,1.439614e+05,-1.630,-52.23,9.024,NaN,99.0,9.0
3,8038,長園科,Industrials,Electrical Equipment & Parts,27.80,-0.39,-0.24,-0.32,-3.37,-3.56,3.479376e+04,-1.380,-638.04,2.054,NaN,99.0,2.0
4,1213,大飲,Consumer Defensive,Beverages - Non-Alcoholic,12.65,391.69,0.27,0.21,656.53,-0.26,2.178304e+04,-1.820,-15.85,1.417,NaN,99.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1870,8112A,至上甲特,Technology,Semiconductors,42.85,NaN,NaN,NaN,NaN,NaN,NaN,7.461,33.70,436.252,5.75,NaN,93.0
1871,8150,南茂,Technology,Semiconductors,36.45,NaN,NaN,NaN,NaN,NaN,NaN,2.440,NaN,30.987,14.88,NaN,NaN
1872,8349A,恒耀甲特,Industrials,Tools & Accessories,48.55,NaN,NaN,NaN,NaN,NaN,NaN,3.247,-17.29,84.455,14.94,NaN,56.0
1873,9110,越南控-DR,Consumer Cyclical,Recreational Vehicles,7.15,NaN,NaN,NaN,NaN,NaN,NaN,-0.130,NaN,0.170,NaN,NaN,NaN



Merged:


,Ticker,Name,RS,1mo:1wk max,3mo:1mo max,6mo:3mo max,9mo:6mo max,Rating (RS),Price,52W pos,MA50,MA200,Volume / VMA50,Sector,Industry,EPS RS,TTM EPS,Rev RS,TTM RPS,TTM PE
0,6144,得利影,286.30,334.85,335.79,196.65,112.51,99,85.1,0.66,104.29,47.32,0.25,Communication Services,Entertainment,-180.29,-0.43,-19.43,5.190,NaN
1,3230,錦明,284.60,287.13,237.68,138.77,105.71,99,48.9,0.74,48.43,22.35,0.61,Technology,Electronic Components,238.62,3.69,-97.13,10.307,15.45
2,8937,合騏,236.90,236.51,205.24,108.28,92.42,99,83.9,1.00,65.57,34.32,0.96,Consumer Cyclical,Recreational Vehicles,267.44,1.12,-64.47,1.649,73.48
3,4510,高鋒,213.59,238.85,227.27,108.10,96.92,99,45.5,0.76,45.86,25.66,0.54,Industrials,Specialty Industrial Machinery,827.80,0.83,-7.12,18.591,57.47
4,4583,台灣精銳,211.83,200.16,166.94,107.82,99.09,99,802.0,0.95,626.31,348.01,1.14,Industrials,Specialty Industrial Machinery,-23.11,13.48,-22.24,34.891,60.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019,3228,金麗科,70.04,72.79,84.79,170.77,240.51,1,223.5,0.24,247.49,361.54,0.81,Technology,Semiconductors,109.62,-0.38,43.08,4.795,NaN
2020,8088,品安,69.34,70.00,83.22,95.05,108.72,1,26.7,0.11,28.27,39.42,0.31,Technology,Computer Hardware,-187.96,0.92,-78.16,20.424,28.59
2021,8477,創業家,67.95,77.51,96.93,95.30,113.32,1,12.7,0.13,12.14,17.37,3.05,Consumer Cyclical,Internet Retail,-199.12,-4.92,-63.84,34.121,NaN
2022,8085,福華,66.27,69.04,93.75,136.45,152.94,1,36.8,0.09,42.01,59.64,1.32,Technology,Electronic Components,-48.91,-1.08,-52.60,4.279,NaN


## Step 2. Filter Stocks

In [17]:
# @title Filter 1. Sorting {"run":"auto"}
cond0 = "(df['RS'] > 100)" # @param ["(df['RS'] > 100)"]
cond1 = "& (df['Rating (RS)'] > 95)" # @param ["& (df['Rating (RS)'] > 95)", "& (df['Rating (RS)'] > 90)", "& (df['Rating (RS)'] > 85)", "& (df['Rating (RS)'] > 80)", ""]
cond2 = "& ((df['EPS RS'] > 0) | (df['Rev RS'] > 0))" # @param ["& ((df['EPS RS'] > 0) | (df['Rev RS'] > 0))", ""]
cond3 = "& (df['EPS RS'] > 0)" # @param ["& (df['EPS RS'] > 0)", "& (df['EPS RS'] > 50)", "& (df['EPS RS'] > 100)", "& (df['EPS RS'] > 200)", "& (df['EPS RS'] > 500)", ""]
cond4 = "& (df['Rev RS'] > 0)" # @param ["& (df['Rev RS'] > 0)", "& (df['Rev RS'] > 5)", "& (df['Rev RS'] > 10)", "& (df['Rev RS'] > 20)", "& (df['Rev RS'] > 50)", "& (df['Rev RS'] > 100)", ""]
cond5 = "& (df['TTM EPS'] > 0)" # @param ["& (df['TTM EPS'] > 0)", "& (df['TTM RPS'] > 0)", "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))", ""]
cond6 = "& (df['52W pos'] > 0.6)" # @param ["& (df['52W pos'] > 0.5)", "& (df['52W pos'] > 0.6)", "& (df['52W pos'] > 0.7)", ""]
cond7 = "" # @param ["& (df['Price'] < 50)", "& (df['Price'] < 100)", "& (df['Price'] < 200)", ""]
sorted_column = "RS" # @param ["RS","1 Week Ago","1 Month Ago","3 Months Ago","6 Months Ago","Price","MA10","MA30","Volume / VMA10","EPS RS", "Rev RS"]
ascending = False # @param {"type":"boolean"}
num_items = 30 # @param [10, 20, 30] {"type":"raw"}

NUM_CONDS = 8
cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))

print_metadata(metadata)
df = df_merge.copy()

df = df[eval(cond)]
df = df.sort_values(by=sorted_column, ascending=ascending)
df_top_f1 = df.head(num_items)
display(df_top_f1)

print_column(df_top_f1, 'Name')
print_column(df_top_f1, 'Ticker')

Source: TWSE+TPEX
Type: stocks
Period: 1y
RS window: 3mo
Date: 20241026


,Ticker,Name,RS,1mo:1wk max,3mo:1mo max,6mo:3mo max,9mo:6mo max,Rating (RS),Price,52W pos,MA50,MA200,Volume / VMA50,Sector,Industry,EPS RS,TTM EPS,Rev RS,TTM RPS,TTM PE
11,1540,喬福,181.12,190.74,183.37,109.51,98.69,98,43.95,0.82,43.52,27.06,0.18,Industrials,Specialty Industrial Machinery,502.41,2.09,9.00,7.516,21.63
20,6640,均華,168.41,197.21,200.69,217.00,207.83,98,730.00,0.65,906.58,539.97,0.85,Technology,Semiconductors,309.51,8.99,36.79,64.467,98.00
24,2543,皇昌,163.93,161.55,152.53,158.92,144.45,98,89.50,1.00,62.52,39.29,0.97,Industrials,Engineering & Construction,480.63,4.98,12.38,46.896,18.67
28,4903,聯光通,160.79,157.01,133.07,123.23,105.97,98,49.40,0.87,36.08,24.35,0.84,Technology,Communication Equipment,181.35,1.15,46.59,7.544,44.43
29,6187,萬潤,160.63,156.91,145.63,153.81,155.67,98,452.00,0.79,421.22,282.94,0.10,Industrials,Specialty Industrial Machinery,547.58,6.21,211.51,32.492,72.46
38,6442,光聖,153.91,202.49,244.39,219.64,155.69,97,485.00,0.88,406.01,254.76,1.19,Technology,Electronic Components,1178.24,6.43,46.11,51.284,68.51
45,4909,新復興,148.94,155.10,149.33,218.35,221.20,97,141.50,0.69,144.13,97.18,0.29,Technology,Electronic Components,1855.67,3.96,317.45,17.321,36.74
48,1235,興泰,146.82,144.62,133.96,98.34,95.88,97,154.50,0.97,136.52,97.06,0.13,Consumer Defensive,Farm Products,49.51,1.88,72.11,1.057,81.38
49,3379,彬台,146.73,194.45,220.63,160.43,102.55,97,35.50,0.65,38.08,25.54,0.33,Industrials,Specialty Industrial Machinery,47.10,0.48,15.07,18.121,79.69
55,3013,晟銘電,145.89,147.36,144.20,149.22,116.47,96,162.50,1.00,126.53,85.83,1.14,Technology,Computer Hardware,56.00,2.08,36.90,38.805,76.44


喬福, 均華, 皇昌, 聯光通, 萬潤, 光聖, 新復興, 興泰, 彬台, 晟銘電, 華景電, 志聖
1540, 6640, 2543, 4903, 6187, 6442, 4909, 1235, 3379, 3013, 6788, 2467


In [18]:
# @title Filter 2. Increasing RS > 0 {"run":"auto"}
cond0 = "(df['RS'] > 100)" # @param ["(df['RS'] > 100)"]
cond1 = "" # @param ["& (df['RS'] > df['1wk:end max'])", ""]
cond2 = "" # @param ["& (df['1wk:end max'] > df['1mo:1wk max'])", ""]
cond3 = "& (df['1mo:1wk max'] > df['3mo:1mo max'])" # @param ["& (df['1mo:1wk max'] > df['3mo:1mo max'])", ""]
cond4 = "& (df['3mo:1mo max'] > df['6mo:3mo max'])" # @param ["& (df['3mo:1mo max'] > df['6mo:3mo max'])", ""]
cond5 = "& (df['6mo:3mo max'] > df['9mo:6mo max'])" # @param ["& (df['6mo:3mo max'] > df['9mo:6mo max'])", ""]
cond6 = "& (df['Rating (RS)'] > 90)" # @param ["& (df['Rating (RS)'] > 95)", "& (df['Rating (RS)'] > 90)", "& (df['Rating (RS)'] > 85)", "& (df['Rating (RS)'] > 80)", ""]
cond7 = "& ((df['EPS RS'] > 0) | (df['Rev RS'] > 0))" # @param ["& ((df['EPS RS'] > 0) | (df['Rev RS'] > 0))", ""]
cond8 = "& (df['EPS RS'] > 0)" # @param ["& (df['EPS RS'] > 0)", "& (df['EPS RS'] > 50)", "& (df['EPS RS'] > 100)", "& (df['EPS RS'] > 200)", "& (df['EPS RS'] > 500)", ""]
cond9 = "& (df['Rev RS'] > 0)" # @param ["& (df['Rev RS'] > 0)", "& (df['Rev RS'] > 5)", "& (df['Rev RS'] > 10)", "& (df['Rev RS'] > 20)", "& (df['Rev RS'] > 50)", "& (df['Rev RS'] > 100)", ""]
cond10 = "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))" # @param ["& (df['TTM EPS'] > 0)", "& (df['TTM RPS'] > 0)", "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))", ""]
cond11 = "& (df['52W pos'] > 0.6)" # @param ["& (df['52W pos'] > 0.5)", "& (df['52W pos'] > 0.6)", "& (df['52W pos'] > 0.7)", ""]
cond12 = "& (df['Price'] < 100)" # @param ["& (df['Price'] < 50)", "& (df['Price'] < 100)", "& (df['Price'] < 200)", ""]
num_items = 20 # @param [10, 20, 30] {"type":"raw"}

NUM_CONDS = 13
cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))

print_metadata(metadata)
df = df_merge.copy()

df = df[eval(cond)]
df_top_f2 = df.head(num_items)
display(df_top_f2)

print_column(df_top_f2, 'Name')
print_column(df_top_f2, 'Ticker')

Source: TWSE+TPEX
Type: stocks
Period: 1y
RS window: 3mo
Date: 20241026


,Ticker,Name,RS,1mo:1wk max,3mo:1mo max,6mo:3mo max,9mo:6mo max,Rating (RS),Price,52W pos,MA50,MA200,Volume / VMA50,Sector,Industry,EPS RS,TTM EPS,Rev RS,TTM RPS,TTM PE
11,1540,喬福,181.12,190.74,183.37,109.51,98.69,98,43.95,0.82,43.52,27.06,0.18,Industrials,Specialty Industrial Machinery,502.41,2.09,9.00,7.516,21.63
28,4903,聯光通,160.79,157.01,133.07,123.23,105.97,98,49.40,0.87,36.08,24.35,0.84,Technology,Communication Equipment,181.35,1.15,46.59,7.544,44.43
101,6807,峰源-KY,129.57,128.72,128.68,124.23,99.18,94,76.50,0.93,74.28,54.33,0.16,Consumer Cyclical,"Furnishings, Fixtures & Appliances",82.86,6.35,0.47,77.857,12.02


喬福, 聯光通, 峰源-KY
1540, 4903, 6807


In [19]:
# @title Filter 3. RS Breakout {"run":"auto"}
base = 130 # @param [100, 105, 110, 115, 120, 130, 160, 200] {"type":"raw"}
cond0 = "(df['RS'] > base)" # @param ["(df['RS'] > base)"]
cond1 = "" # @param ["& (df['1wk:end max'] < base)",""]
cond2 = "" # @param ["& (df['1mo:1wk max'] < base)",""]
cond3 = "& (df['3mo:1mo max'] < base)" # @param ["& (df['3mo:1mo max'] < base)",""]
cond4 = "& (df['6mo:3mo max'] < base)" # @param ["& (df['6mo:3mo max'] < base)",""]
cond5 = "& (df['9mo:6mo max'] < base)" # @param ["& (df['9mo:6mo max'] < base)",""]
cond6 = "" # @param ["& (df['Rating (RS)'] > 95)", "& (df['Rating (RS)'] > 90)", "& (df['Rating (RS)'] > 85)", "& (df['Rating (RS)'] > 80)", ""]
cond7 = "& (df['Volume / VMA50'] > 2)" # @param ["& (df['Volume / VMA50'] > 1.5)", "& (df['Volume / VMA50'] > 2)", "& (df['Volume / VMA50'] > 3)", "& (df['Volume / VMA50'] > 4)", ""]
cond8 = "& (df['Price'] > df['MA200'])" # @param ["& (df['Price'] > df['MA50'])", "& (df['Price'] > df['MA200'])", ""]
cond9 = "& (df['MA50'] > df['MA200'])" # @param ["& (df['MA50'] > df['MA200'])",""]
cond10 = "& ((df['EPS RS'] > 0) | (df['Rev RS'] > 0))" # @param ["& ((df['EPS RS'] > 0) | (df['Rev RS'] > 0))", ""]
cond11 = "& (df['EPS RS'] > 0)" # @param ["& (df['EPS RS'] > 0)", "& (df['EPS RS'] > 50)", "& (df['EPS RS'] > 100)", "& (df['EPS RS'] > 200)", "& (df['EPS RS'] > 500)", ""]
cond12 = "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))" # @param ["& (df['TTM EPS'] > 0)", "& (df['TTM RPS'] > 0)", "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))", ""]
cond13 = "" # @param ["& (df['Rev RS'] > 0)", "& (df['Rev RS'] > 5)", "& (df['Rev RS'] > 10)", "& (df['Rev RS'] > 20)", "& (df['Rev RS'] > 50)", "& (df['Rev RS'] > 100)", ""]
cond14 = "& (df['52W pos'] > 0.7)" # @param ["& (df['52W pos'] > 0.5)", "& (df['52W pos'] > 0.6)", "& (df['52W pos'] > 0.7)", ""]
cond15 = "" # @param ["& (df['Price'] < 50)", "& (df['Price'] < 100)", "& (df['Price'] < 200)", "& (df['Price'] < 500)", ""]
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

NUM_CONDS = 16
cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))

print_metadata(metadata)
df = df_merge.copy()

df = df[eval(cond)]
df_top_f3 = df.head(num_items)
display(df_top_f3)

print_column(df_top_f3, 'Name')
print_column(df_top_f3, 'Ticker')

Source: TWSE+TPEX
Type: stocks
Period: 1y
RS window: 3mo
Date: 20241026


,Ticker,Name,RS,1mo:1wk max,3mo:1mo max,6mo:3mo max,9mo:6mo max,Rating (RS),Price,52W pos,MA50,MA200,Volume / VMA50,Sector,Industry,EPS RS,TTM EPS,Rev RS,TTM RPS,TTM PE


In [20]:
# @title Filter 4. Groupby "Industry" {"run":"auto"}
cond0 = "(df['RS'] > 100)" # @param ["(df['RS'] > 100)"]
cond1 = "& (df['Rating (RS)'] > 95)" # @param ["& (df['Rating (RS)'] > 95)", "& (df['Rating (RS)'] > 90)", "& (df['Rating (RS)'] > 85)", "& (df['Rating (RS)'] > 80)", ""]
num_items = 2 # @param [1, 2, 3, 4, 5] {"type":"raw"}

NUM_CONDS = 2

from vistock.ranking_utils import append_ratings, groupby_industry

print_metadata(metadata)
stock_df = df_merge.copy()

# Filter out rows with NaN in the 'Ticker' column
#stock_df = stock_df[stock_df['Ticker'].notna()]

rs_columns = ['RS',
              '1mo:1wk max', '3mo:1mo max', '6mo:3mo max', '9mo:6mo max']
if 'Name' in stock_df.columns:
    columns = ['Sector', 'Ticker', 'Name'] + rs_columns
else:
    columns = ['Sector', 'Ticker'] + rs_columns
df = groupby_industry(stock_df, columns, key='RS')

df = df.sort_values(by='RS', ascending=False).reset_index(drop=True)
rating_columns = ['Rating (RS)', 'Rating (1M:1W)', 'Rating (3M:1M)',
                  'Rating (6M:3M)', 'Rating (9M:6M)']
df = append_ratings(df, rs_columns, rating_columns)

cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))
df = df[eval(cond)]
df_top_f4 = df.head(num_items)
display(df_top_f4)

print_column(df_top_f4, 'Name')
print_column(df_top_f4, 'Ticker')

Source: TWSE+TPEX
Type: stocks
Period: 1y
RS window: 3mo
Date: 20241026


,Industry,Sector,Ticker,Name,RS,1mo:1wk max,3mo:1mo max,6mo:3mo max,9mo:6mo max,Rating (RS),Rating (1M:1W),Rating (3M:1M),Rating (6M:3M),Rating (9M:6M)
0,Recreational Vehicles,Consumer Cyclical,"8937,9110,8478","合騏,越南控-DR,東哥遊艇",138.42,139.30,130.61,98.87,94.71,99,98,97,49,16
1,Industrial Distribution,Industrials,"8374,2373,3114","羅昇,震旦行,好德",128.59,151.46,164.23,132.11,94.94,98,99,99,99,18


合騏,越南控-DR,東哥遊艇, 羅昇,震旦行,好德
8937,9110,8478, 8374,2373,3114


In [21]:
# @title Filter 5. Financial {"run":"auto"}
cond0 = "((df['EPS RS'] > 0) | (df['Rev RS'] > 0))" # @param ["((df['EPS RS'] > 0) | (df['Rev RS'] > 0))", ""]
cond1 = "& (df['EPS RS'] > 0)" # @param ["& (df['EPS RS'] > 0)", "& (df['EPS RS'] > 50)", "& (df['EPS RS'] > 100)", "& (df['EPS RS'] > 200)", "& (df['EPS RS'] > 500)", ""]
cond2 = "& (df['Rev RS'] > 0)" # @param ["& (df['Rev RS'] > 0)", "& (df['Rev RS'] > 5)", "& (df['Rev RS'] > 10)", "& (df['Rev RS'] > 20)", "& (df['Rev RS'] > 50)", "& (df['Rev RS'] > 100)", ""]
cond3 = "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))" # @param ["& (df['TTM EPS'] > 0)", "& (df['TTM RPS'] > 0)", "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))", ""]
cond4 = "& (df['EPS YoY (%)'] > 0)" # @param ["& (df['EPS YoY (%)'] > 0)", "& ((df['EPS YoY (%)'] > 0) & (df['YoY 2Q Algo (%)'] > 0))", ""]
cond5 = "& ((df['EPS QoQ (%)'] > 0) & (df['QoQ 2Q Algo (%)'] > 0) & (df['QoQ 3Q Algo (%)'] > 0))" # @param ["& (df['EPS QoQ (%)'] > 0)", "& ((df['EPS QoQ (%)'] > 0) & (df['QoQ 2Q Algo (%)'] > 0))", "& ((df['EPS QoQ (%)'] > 0) & (df['QoQ 2Q Algo (%)'] > 0) & (df['QoQ 3Q Algo (%)'] > 0))", ""]
cond6 = "" # @param ["& (df['Price'] < 50)", "& (df['Price'] < 100)", "& (df['Price'] < 200)", ""]
cond7 = "& (df['Sector'] == 'Technology')" # @param ["& (df['Sector'] == 'Technology')", "& (df['Sector'] == 'Energy')", ""]
sorted_column = "EPS RS" # @param ["EPS RS", "Rev RS"]
ascending = False # @param {"type":"boolean"}
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

NUM_CONDS = 8
cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))

df = df_fin.copy()

import numpy as np

df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df[eval(cond)]
df = df.sort_values(by=sorted_column, ascending=ascending)
df_top_f5 = df.head(num_items)
display(df_top_f5)

print_column(df_top_f5, 'Name')
print_column(df_top_f5, 'Ticker')

,Ticker,Name,Sector,Industry,Price,EPS QoQ (%),QoQ 2Q Algo (%),QoQ 3Q Algo (%),EPS YoY (%),YoY 2Q Algo (%),EPS RS,TTM EPS,Rev RS,TTM RPS,TTM PE,Rating (EPS RS),Rating (Rev RS)
65,6442,光聖,Technology,Electronic Components,432.00,0.90,0.26,0.36,2.06,33.40,1178.24,6.43,46.11,51.284,68.51,96.0,94.0
69,2455,全新,Technology,Semiconductor Equipment & Materials,165.50,0.05,0.33,0.37,17.17,NaN,1086.83,1.61,55.81,18.487,101.24,95.0,95.0
72,6103,合邦,Technology,Semiconductors,51.30,0.96,0.33,0.50,28.50,2.08,1075.09,0.58,2.28,3.296,90.00,95.0,84.0
179,8210,勤誠,Technology,Computer Hardware,294.50,0.24,0.19,0.23,0.82,12.17,376.04,13.29,8.05,111.900,21.93,89.0,88.0
197,6716,應廣,Technology,Semiconductors,98.90,0.30,0.26,0.36,2.46,10.00,340.91,4.86,1.91,43.535,19.98,88.0,84.0
250,6735,美達科技,Technology,Semiconductor Equipment & Materials,77.40,0.16,0.77,3.33,8.27,3.09,239.76,2.10,58.15,8.663,36.00,86.0,95.0
272,6216,居易,Technology,Communication Equipment,44.75,0.47,0.08,0.09,0.51,7.11,196.14,2.28,17.20,8.421,19.32,84.0,90.0
307,3591,艾笛森,Technology,Electronic Components,28.10,0.68,0.73,0.10,2.20,NaN,154.20,0.11,4.26,16.155,244.55,83.0,86.0
321,6877,鏵友益,Technology,Semiconductor Equipment & Materials,74.00,10.88,0.45,0.83,11.56,-4.13,130.41,0.17,33.66,10.472,428.82,82.0,93.0
397,6274,台燿,Technology,Electronic Components,168.50,0.54,0.10,0.11,1.55,3.16,87.56,6.88,7.37,69.014,23.98,78.0,88.0


光聖, 全新, 合邦, 勤誠, 應廣, 美達科技, 居易, 艾笛森, 鏵友益, 台燿
6442, 2455, 6103, 8210, 6716, 6735, 6216, 3591, 6877, 6274


## Step 3. Visualize Filtered Stocks

In [22]:
# @title Plot 1. IBD Relative Strength Comparison {"run":"auto"}
source = "Filter 5. Financial" # @param ["Filter 1. Sorting", "Filter 2. Increasing RS","Filter 3. RS Breakout", "Filter 4. Groupby Industry", "Filter 5. Financial", "Major Global Stock Indices", "Sector Indices"]
backend = "mplfinance" # @param ["mplfinance","Plotly"]

import matplotlib.pyplot as plt
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output

MAX_STOCK_SELECTION = 10
N_COLS = 8  # the number of columns of the grid layout for checkboxes

symbols = {
    'Filter 1. Sorting': lambda: tickers_from_df(df_top_f1),
    'Filter 2. Increasing RS': lambda: tickers_from_df(df_top_f2),
    'Filter 3. RS Breakout': lambda: tickers_from_df(df_top_f3),
    'Filter 4. Groupby Industry': lambda: tickers_from_df(df_top_f4),
    'Filter 5. Financial': lambda: tickers_from_df(df_top_f5),
    'Major Global Stock Indices': major_indices,
    'Sector Indices': sector_indices,
}[source]()

checkboxes = cbs_create(symbols, MAX_STOCK_SELECTION)
checkbox_grid = cbs_with_grid(checkboxes, N_COLS)
btn_unselect_all = widgets.Button(description="Unselect All")
btn_unselect_all.on_click(lambda b: cbs_unselect_all(checkboxes))
btn_select_top = widgets.Button(description="Select Top 10")
btn_select_top.on_click(lambda b: cbs_select_top(checkboxes, MAX_STOCK_SELECTION))

period_dropdown = create_period_dropdown(metadata['Period'])
interval_dropown = create_interval_dropdown('1d')
rs_window_dropdown = create_rs_window_dropdown(metadata['RS window'])

cmp_theme_dropdown = {
    'mplfinance': create_style_dropdown('Comparison Theme:', 'charles'),
    'Plotly': create_template_dropdown('Comparison Theme:', 'plotly_dark'),
}[backend]
btn_plot = widgets.Button(description="Generate Plot")

out_msg, out_fig = widgets.Output(), widgets.Output()

ui = widgets.VBox([
    checkbox_grid,
    widgets.HBox([btn_unselect_all, btn_select_top]),
    widgets.VBox([period_dropdown, interval_dropown, rs_window_dropdown]),
    widgets.VBox([cmp_theme_dropdown, btn_plot]),
    out_msg, out_fig
])
display(ui)

import vistock.mpl as mpl
import vistock.plotly as ply

rs_cmp = {
    'mplfinance': mpl.ibd_rs_cmp,
    'Plotly': ply.ibd_rs_cmp,
}[backend]

def on_checkbox_change(change):
    selected_count = sum([cb.value for cb in checkboxes])
    if selected_count > MAX_STOCK_SELECTION:
        # Uncheck the last checked box if selection exceeds limit
        changed_checkbox = change['owner']
        changed_checkbox.value = False
        with out_msg:
            out_fig.clear_output()
            print(f"Only {MAX_STOCK_SELECTION} stocks can be selected at most.")

# Bind the checkbox change event to the function
for checkbox in checkboxes:
    checkbox.observe(on_checkbox_change, names='value')

def on_plot_click(b):
    symbols = cbs_get_selected(checkboxes)
    if not symbols:
        with out_msg:
            out_fig.clear_output()
            print("No stocks selected. Please select at least one stock.")
        return
    with out_fig:
        out_msg.clear_output()
        clear_output()
        interval = interval_dropown.value
        period = period_dropdown.value
        rs_window = rs_window_dropdown.value
        if rs_cmp is mpl.ibd_rs_cmp:
            rs_cmp.plot(symbols, interval=interval, period=period,
                        rs_window=rs_window,
                        style=cmp_theme_dropdown.value,
                        color_cycle=plt.cm.Paired.colors)
        else: # Plotly
            rs_cmp.plot(symbols, interval=interval, period=period,
                        rs_window=rs_window,
                        template=cmp_theme_dropdown.value,
                        colorway=px.colors.qualitative.Set3)

btn_plot.on_click(on_plot_click)

if backend == 'Plotly':
    enable_plotly_in_cell()


In [ ]:
# @title Plot 2. IBD Stock Chart {"run":"auto"}
source = "Filter 5. Financial" # @param ["Filter 1. Sorting", "Filter 2. Increasing RS","Filter 3. RS Breakout","Filter 4. Groupby Industry","Filter 5. Financial","Major Global Stock Indices", "U.S. Listed Stocks", "Taiwan Stocks"]
backend = "mplfinance" # @param ["mplfinance","Plotly"]

import functools as ft
import ipywidgets as widgets
from IPython.display import display, clear_output
import yfinance as yf

import vistock.stock_indices as si
import vistock.tw as tw

symbols = {
    'Filter 1. Sorting': lambda: tickers_from_df(df_top_f1),
    'Filter 2. Increasing RS': lambda: tickers_from_df(df_top_f2),
    'Filter 3. RS Breakout': lambda: tickers_from_df(df_top_f3),
    'Filter 4. Groupby Industry': lambda: tickers_from_df(df_top_f4),
    'Filter 5. Financial': lambda: tickers_from_df(df_top_f5),
    'Major Global Stock Indices': major_indices,
    'U.S. Listed Stocks': ft.partial(si.get_tickers, 'USLS'),
    'Taiwan Stocks': ft.partial(si.get_tickers, 'TWSE+TPEX+ESB'),
}[source]()
if source == 'Taiwan Stocks':
    symbols = [f"{tw_stock_name(s)} {s}" for s in symbols]

dropdowns, layout = create_search_dropdowns(symbols, 1)

period_dropdown2 = create_period_dropdown(metadata['Period'])
interval_dropown2 = create_interval_dropdown('1d')
rs_window_dropdown2 = create_rs_window_dropdown(metadata['RS window'])

stock_theme_dropdown = {
    'mplfinance': create_style_dropdown('Stock Theme:', 'yahoo'),
    'Plotly': create_template_dropdown('Stock Theme:', 'plotly'),
}[backend]

btn_plot_prc = widgets.Button(description="Price/RS/Volume Chart",
                              layout=widgets.Layout(width='168px'))
btn_plot_fin = widgets.Button(description="Financial Chart",
                              layout=widgets.Layout(width='168px'))
btn_report_q = widgets.Button(description="Quarterly Report",
                              layout=widgets.Layout(width='168px'))
btn_report_a = widgets.Button(description="Annual Report",
                              layout=widgets.Layout(width='168px'))
btn_clear_last = widgets.Button(description="Clear Last")
btn_clear_all = widgets.Button(description="Clear All")
#outputs = widgets.VBox()

ui = widgets.VBox([
    layout,
    period_dropdown2, interval_dropown2, rs_window_dropdown2, stock_theme_dropdown,
    widgets.HBox([btn_plot_prc, btn_plot_fin]),
    widgets.HBox([btn_report_q, btn_report_a]),
    widgets.HBox([btn_clear_last, btn_clear_all]),
    outputs
])
display(ui)

import vistock.mpl as mpl
import vistock.plotly as ply

stock_chart = {
    'mplfinance': mpl.ibd_rs,
    'Plotly': ply.ibd_rs,
}[backend]

def get_symbols():
    symbols = get_dropdowns_selected_options(dropdowns)
    if source == 'Taiwan Stocks':
        symbols = [s.split()[0] for s in symbols]
    return symbols

def on_plot_prc_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        interval = interval_dropown2.value
        period = period_dropdown2.value
        rs_window = rs_window_dropdown2.value
        if not selected:
            print("No Stock Selected!")
        elif stock_chart is mpl.ibd_rs:
            stock_chart.plot(selected[0], interval=interval, period=period,
                             rs_window=rs_window,
                             style=stock_theme_dropdown.value,
                             legend_loc='upper left')
        else: # Plotly
            stock_chart.plot(selected[0], interval=interval, period=period,
                             rs_window=rs_window,
                             template=stock_theme_dropdown.value)
    outputs.children = (new_output,) + outputs.children

import vistock.mpl.financials as mpl_fin
import vistock.plotly.financials as ply_fin

fin_chart = {
    'mplfinance': mpl_fin,
    'Plotly': ply_fin,
}[backend]

def on_plot_fin_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        if not selected:
            print("No Stock Selected!")
        elif fin_chart is mpl_fin:
            fin_chart.plot(selected[0], style=stock_theme_dropdown.value)
        else: # Plotly
            fin_chart.plot(selected[0], template=stock_theme_dropdown.value)
    outputs.children = (new_output,) + outputs.children

def on_report_q_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        if not selected:
            print("No Stock Selected!")
        else:
            symbol = tw.as_yfinance(selected[0])
            ticker = tw.as_yfinance(symbol)
            print(f"\n{symbol} Quarterly Financials:")
            display(yf.Ticker(ticker).quarterly_financials)
    outputs.children = (new_output,) + outputs.children

def on_report_a_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        if not selected:
            print("No Stock Selected!")
        else:
            symbol = tw.as_yfinance(selected[0])
            ticker = tw.as_yfinance(symbol)
            print(f"\n{symbol} Annual Financials:")
            display(yf.Ticker(ticker).financials)
    outputs.children = (new_output,) + outputs.children

def on_clear_last_click(b):
    if outputs.children:
        children = list(outputs.children)
        children.pop(0)
        outputs.children = tuple(children)

def on_clear_all_click(b):
    outputs.children = ()

btn_plot_prc.on_click(on_plot_prc_click)
btn_plot_fin.on_click(on_plot_fin_click)
btn_report_q.on_click(on_report_q_click)
btn_report_a.on_click(on_report_a_click)
btn_clear_last.on_click(on_clear_last_click)
btn_clear_all.on_click(on_clear_all_click)

if backend == 'Plotly':
    enable_plotly_in_cell()
